<a href="https://colab.research.google.com/github/ccloveak/Study_Notes/blob/master/SSH%E5%8E%9F%E7%90%86%E4%B8%8E%E8%BF%90%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.什么是SSH？

SSH是一种网络协议，用于计算机之间的加密登录

如果一个用户从本地计算机，使用SSH协议登录另一台远程计算机，我们就可以认为，这种登录是安全的，即使被中途截获，密码也不会泄露。  


#2.最基本的用法

1. 以用户名user，登录远程主机host


```
　　$ ssh user@host
```
2. 如果本地用户名与远程用户名一致，登录时可以省略用户名


```
$ ssh host
```
3. SSH的默认端口是22，也就是说，你的登录请求会送进远程主机的22端口。使用p参数，可以修改这个端口。ssh直接连接远程主机的2222端口。


```
　　$ ssh -p 2222 user@host
```






#3. 中间人攻击

SSH之所以能够保证安全，原因在于它采用了公钥加密。  
整个过程：

1.   远程主机收到用户的登录请求，把自己的公钥发给用户。
2. 用户使用这个公钥，将登录密码加密后，发送回来。 
3.  远程主机用自己的私钥，解密登录密码，如果密码正确，就同意用户登录。

实施的时候存在一个风险：如果有人截获了登录请求，然后冒充远程主机，将伪造的公钥发给用户，那么用户很难辨别真伪。    
"中间人攻击"（Man-in-the-middle attack）：如果攻击者插在用户与远程主机之间（比如在公共的wifi区域），用伪造的公钥，获取用户的登录密码。再用这个密码登录远程主机，那么SSH的安全机制就荡然无存了。

#4. 口令登录

如果你是第一次登录对方主机，系统会出现下面的提示：  


```
$ ssh user@host

　　The authenticity of host 'host (12.18.429.21)' can't be established.

　　RSA key fingerprint is 98:2e:d7:e0:de:9f:ac:67:28:c2:42:2d:37:16:58:4d.

　　Are you sure you want to continue connecting (yes/no)?
```
假定经过风险衡量以后，用户决定接受这个远程主机的公钥。


```
Are you sure you want to continue connecting (yes/no)? yes
```
系统会出现一句提示，表示host主机已经得到认可。



```
Warning: Permanently added 'host,12.18.429.21' (RSA) to the list of known hosts.
```
然后，会要求输入密码


```
　Password: (enter password)
```

如果密码正确，就可以登录了  

当远程主机的公钥被接受以后，它就会被保存在文件$HOME/.ssh/known_hosts之中。下次再连接这台主机，系统就会认出它的公钥已经保存在本地了，从而跳过警告部分，直接提示输入密码。

每个SSH用户都有自己的known_hosts文件，此外系统也有一个这样的文件，通常是/etc/ssh/ssh_known_hosts，保存一些对所有用户都可信赖的远程主机的公钥。  









#5. 公钥登录

SSH还提供了公钥登录，可以省去输入密码的步骤。

用户将自己的公钥储存在远程主机上。登录的时候，远程主机会向用户发送一段随机字符串，用户用自己的私钥加密后，再发回来。远程主机用事先储存的公钥进行解密，如果成功，就证明用户是可信的，直接允许登录shell，不再要求密码。



要求用户必须提供自己的公钥。如果没有现成的，可以直接用ssh-keygen生成一个： 


```
$ ssh-keygen
```
一路回车。  
要不要对私钥设置口令（passphrase）：如果担心私钥的安全，这里可以设置一个。  

输入下面的命令，将公钥传送到远程主机host上面  


```
$ ssh-copy-id user@host
```

如果还是不行，就打开远程主机的/etc/ssh/sshd_config这个文件，检查下面几行前面"#"注释是否取掉。


```
RSAAuthentication yes
　　PubkeyAuthentication yes
　　AuthorizedKeysFile .ssh/authorized_keys
```  
重启远程主机的ssh服务  


```
　　// ubuntu系统
　　service ssh restart

　　// debian系统
　　/etc/init.d/ssh restart
```





#6. authorized_keys文件

远程主机将用户的公钥，保存在登录后的用户主目录的`$HOME/.ssh/authorized_keys`文件中。公钥就是一段字符串，只要把它追加在`authorized_keys`文件的末尾就行了。

**解释公钥的保存过程： **   


```
$ ssh user@host 'mkdir -p .ssh && cat >> .ssh/authorized_keys' < ~/.ssh/id_rsa.pub


``` 


1.   `"$ ssh user@host"`，表示登录远程主机
2. 单引号中的`mkdir .ssh && cat >> .ssh/authorized_keys`，表示登录后在远程shell上执行的命令：
3. `"$ mkdir -p .ssh"`的作用是，如果用户主目录中的.ssh目录不存在，就创建一个；
4.  `'cat >> .ssh/authorized_keys' < ~/.ssh/id_rsa.pub`的作用是，将本地的公钥文件`~/.ssh/id_rsa.pub`重定向追加到远程文件`authorized_keys`的末尾。







#7. 远程操作

SSH不仅可以用于远程主机登录，还可以直接在远程主机上执行操作。



```
　　$ ssh user@host 'mkdir -p .ssh && cat >> .ssh/authorized_keys' < ~/.ssh/id_rsa.pub
```
单引号中间的部分，表示在远程主机上执行的操作；后面的输入重定向，表示数据通过SSH传向远程主机。  
这就是说，SSH可以在用户和远程主机之间，建立命令和数据的传输通道，因此很多事情都可以通过SSH来完成。


将`$HOME/src/`目录下面的所有文件，复制到远程主机的`$HOME/src/`目录。  


```
$ cd && tar czv src | ssh user@host 'tar xz'
```  
将远程主机`$HOME/src/`目录下面的所有文件，复制到用户的当前目录。  


```
$ ssh user@host 'tar cz src' | tar xzv
```
查看远程主机是否运行进程httpd  


```
　　$ ssh user@host 'ps ax | grep [h]ttpd'
```






#8. 绑定本地端口

既然SSH可以传送数据，可以让那些不加密的网络连接，全部改走SSH连接，从而提高安全性。

假定要让8080端口的数据，都通过SSH传向远程主机，命令就这样写  


```
$ ssh -D 8080 user@host
```
SSH会建立一个socket，去监听本地的8080端口。一旦有数据传向那个端口，就自动把它转移到SSH连接上面，发往远程主机。可以想象，如果8080端口原来是一个不加密端口，现在将变成一个加密端口。


#9. 本地端口转发

绑定本地端口还不够，还必须指定数据传送的目标主机，从而形成点对点的"端口转发"。  
为了区别后文的"远程端口转发"，我们把这种情况称为"本地端口转发"（Local forwarding）  

假定host1是本地主机，host2是远程主机。由于种种原因，这两台主机之间无法连通。但是，另外还有一台host3，可以同时连通前面两台主机。   
很自然的想法就是，通过host3，将host1连上host2。  
我们在host1执行下面的命令  


```
　$ ssh -L 2121:host2:21 host3
```

命令中的L参数一共接受三个值，分别是"本地端口:目标主机:目标主机端口"  
指定SSH绑定本地端口2121，然后指定host3将所有的数据，转发到目标主机host2的21端口（假定host2运行FTP，默认端口为21）  
只要连接host1的2121端口，就等于连上了host2的21端口  


```
$ ftp localhost:2121
```

"本地端口转发"使得host1和host3之间仿佛形成一个数据传输的秘密隧道，因此又被称为"SSH隧道"。

一个比较有趣的例子  


```
　$ ssh -L 5900:localhost:5900 host3
```

将本机的5900端口绑定host3的5900端口（这里的localhost指的是host3，因为目标主机是相对host3而言的）  


另一个例子是通过host3的端口转发，ssh登录host2  


```
$ ssh -L 9001:host2:22 host3
```
这时，只要ssh登录本机的9001端口，就相当于登录host2了  


```
$ ssh -p 9001 localhost
```
上面的-p参数表示指定登录端口



# 10. 远程端口转发

"本地端口转发"是指绑定本地端口的转发  
"远程端口转发"（remote forwarding）是指绑定远程端口的转发

接着看上面那个例子，host1与host2之间无法连通，必须借助host3转发。但是，特殊情况出现了，host3是一台内网机器，它可以连接外网的host1，但是反过来就不行，外网的host1连不上内网的host3。  
解决办法是，既然host3可以连host1，那么就从host3上建立与host1的SSH连接，然后在host1上使用这条连接就可以了。

在host3执行下面的命令：  


```
$ ssh -R 2121:host2:21 host1
```
R参数也是接受三个值，分别是"远程主机端口:目标主机:目标主机端口"。  
让host1监听它自己的2121端口，然后将所有数据经由host3，转发到host2的21端口。  
由于对于host3来说，host1是远程主机，所以这种情况就被称为"远程端口绑定"


绑定之后，我们在host1就可以连接host2了：  


```
$ ftp localhost:2121
```
"远程端口转发"的前提条件是，host1和host3两台主机都有sshD和ssh客户端


#11. SSH的其他参数

N参数，表示只连接远程主机，不打开远程shell；T参数，表示不为这个连接分配TTY。这个两个参数可以放在一起用，代表这个SSH连接只用来传数据，不执行远程操作。  


```
$ ssh -NT -D 8080 host
```



f参数，表示SSH连接成功后，转入后台运行。这样一来，你就可以在不中断SSH连接的情况下，在本地shell中执行其他操作  


```
f参数，表示SSH连接成功后，转入后台运行。这样一来，你就可以在不中断SSH连接的情况下，在本地shell中执行其他操作
```



#12. 参考文献

Jianing YANG：[ssh隧道技术简介  ](https://wenku.baidu.com/view/f1b4471bbe23482fb4da4ca8.html)  
IBM developerWorks: [实战 SSH 端口转发](https://www.ibm.com/developerworks/cn/linux/l-cn-sshforward/index.html)